In [152]:
# Initial imports
import os
import json
import requests
import pandas as pd
from pathlib import Path
import quandl
from dotenv import load_dotenv

%matplotlib inline

In [153]:
load_dotenv()

True

In [154]:
# Set API key from env variables
quandl.ApiConfig.api_key = os.getenv("QUANDL_API")

## Indicators
The indicators table explains how the housing data has been categorized

In [155]:
# Download housing data from quadl api
indicator_df = quandl.get_table("ZILLOW/INDICATORS")

In [156]:
# Show indicators dataframe
indicator_df.head(20)

,indicator_id,indicator,category
None,,,
0,ZSFH,ZHVI Single-Family Homes Time Series ($),Home values
1,ZCON,ZHVI Condo/Co-op Time Series ($),Home values
2,ZATT,ZHVI All Homes- Top Tier Time Series ($),Home values
3,ZALL,"ZHVI All Homes (SFR, Condo/Co-op) Time Series ($)",Home values
4,ZABT,ZHVI All Homes- Bottom Tier Time Series ($),Home values
5,Z5BR,ZHVI 5+ Bedroom Time Series ($),Home values
6,Z4BR,ZHVI 4-Bedroom Time Series ($),Home values
7,Z3BR,ZHVI 3-Bedroom Time Series ($),Home values
8,Z2BR,ZHVI 2-Bedroom Time Series ($),Home values


## Regions
The regions table explains the different region types for the housing data

In [157]:
# Show regions dataframe
regions = quandl.get_table("ZILLOW/REGIONS", paginate=True)
regions.head()

,region_id,region_type,region
None,,,
0,99999,zip,98847; WA; Wenatchee; Chelan County; Peshastin
1,99998,zip,98846; WA; Okanogan County; Pateros
2,99997,zip,98845; WA; Wenatchee; Douglas County; Palisades
3,99996,zip,98844; WA; Okanogan County; Oroville
4,99995,zip,98843; WA; Wenatchee; Douglas County; Orondo


## Data Table
The data table contains all the housing data by indicator and region.

The downloaded zip is extracted to a location outside of the repository due to it's large size.

In [158]:
# -- Code commented out as it only needs to be run a single time

# -------
# data = quandl.export_table('ZILLOW/DATA', filename="../../Repositories/local/zdata.zip")
# with zipfile.ZipFile('../../Repositories/local/zdata.zip', 'r') as zip_ref:
#    zip_ref.extractall('../../Repositories/local/')
# os.rename('../../Repositories/local/ZILLOW_DATA_962c837a6ccefddddf190101e0bafdaf.csv', '../../Repositories/local/zdata.csv')
# Read extracted zip file into a dataframe
zdata = pd.read_csv('../../Repositories/local/zdata.csv')
# -------

In [159]:
zdata.shape

(127237469, 4)

In [160]:
regions.region_type.value_counts()

zip       31172
city      27471
neigh     16711
county     2887
metro       915
state        51
Name: region_type, dtype: int64

## Extract metro codes to list

In [161]:
# Casting the list values as int for use to make splits from the region_id column
metros = regions[regions.region_type == 'metro']
metros.tail()

,region_id,region_type,region
None,,,
51628,394300,metro,"Ada, OK; OK"
51630,394299,metro,"Abilene, TX; TX"
51631,394298,metro,"Aberdeen, WA; WA"
51632,394297,metro,"Aberdeen, SD; SD"
78827,102001,metro,United States


In [162]:
metro_codes = [int(i) for i in metros.region_id.tolist()]
metro_codes[:5]

[753929, 753928, 753927, 753926, 753925]

## Extract home value codes to list

In [163]:
home_value_codes = []
for index, row in indicator_df.iterrows():
    if row[2] == "Home values":
        home_value_codes.append(row[0])
home_value_codes

['ZSFH',
 'ZCON',
 'ZATT',
 'ZALL',
 'ZABT',
 'Z5BR',
 'Z4BR',
 'Z3BR',
 'Z2BR',
 'Z1BR']

In [164]:
del home_value_codes[0]
del home_value_codes[1:4]
home_value_codes

['ZCON', 'Z5BR', 'Z4BR', 'Z3BR', 'Z2BR', 'Z1BR']

## Home values used to create a slice of zdata

In [165]:
# Dataframe containing all homes from 1BR to 5BR and Condos
homes = zdata[zdata.indicator_id.isin(home_value_codes)]
homes

,indicator_id,region_id,date,value
11667,ZCON,9,2020-10-31,537729.0
11960,ZCON,54,2020-10-31,185435.0
12253,ZCON,43,2020-10-31,580421.0
12546,ZCON,14,2020-10-31,196766.0
12839,ZCON,21,2020-10-31,202002.0
...,...,...,...,...
127237464,Z4BR,49589,2020-02-29,103928.0
127237465,Z4BR,49589,2020-03-31,105018.0
127237466,Z4BR,49589,2020-04-30,105616.0
127237467,Z4BR,49589,2020-06-30,105040.0


In [166]:
majors_list = ['New York, NY','Los Angeles-Long Beach-Anaheim, CA','Chicago, IL','Houston, TX','Phoenix, AZ','Philadelphia, PA','San Antonio, TX',
               'San Diego, CA','Dallas-Fort Worth, TX','Austin, TX','San Jose, CA','Jacksonville, FL','Columbus, OH','Charlotte, NC',
               'Indianapolis, IN','San Francisco, CA','Seattle, WA','Denver, CO','Washington, DC','Boston, MA','El Paso, TX',
               'Nashville, TN','Oklahoma City, OK','Las Vegas, NV','Detroit, MI','Portland, OR','Memphis, TN','Louisville-Jefferson County, KY',
               'Milwaukee, WI','Baltimore, MD']
majors_list.sort()
majors_list

['Austin, TX',
 'Baltimore, MD',
 'Boston, MA',
 'Charlotte, NC',
 'Chicago, IL',
 'Columbus, OH',
 'Dallas-Fort Worth, TX',
 'Denver, CO',
 'Detroit, MI',
 'El Paso, TX',
 'Houston, TX',
 'Indianapolis, IN',
 'Jacksonville, FL',
 'Las Vegas, NV',
 'Los Angeles-Long Beach-Anaheim, CA',
 'Louisville-Jefferson County, KY',
 'Memphis, TN',
 'Milwaukee, WI',
 'Nashville, TN',
 'New York, NY',
 'Oklahoma City, OK',
 'Philadelphia, PA',
 'Phoenix, AZ',
 'Portland, OR',
 'San Antonio, TX',
 'San Diego, CA',
 'San Francisco, CA',
 'San Jose, CA',
 'Seattle, WA',
 'Washington, DC']

In [167]:
finder = metros.loc[metros['region'].str.contains("Louisville", case=False)]
finder

,region_id,region_type,region
None,,,
51142,394807,metro,"Louisville-Jefferson County, KY"


In [168]:
major_metros = metros[metros.region.isin(majors_list)]
major_metros.sort_values('region', inplace=True)
major_metros = major_metros.drop('region_type', axis=1)
major_metros.region_id = major_metros.region_id.astype(str)
major_metros

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,region_id,region
None,,
51577,394355,"Austin, TX"
51574,394358,"Baltimore, MD"
51527,394404,"Boston, MA"
51474,394458,"Charlotte, NC"
51470,394463,"Chicago, IL"
51441,394492,"Columbus, OH"
51420,394514,"Dallas-Fort Worth, TX"
51403,394530,"Denver, CO"
51401,394532,"Detroit, MI"


## Metro codes used to slice homes data

In [169]:
metro_condo = homes[homes.region_id.isin(metro_codes)]
metro_condo = homes[homes.indicator_id == "ZCON"]
metro_condo.region_id = metro_condo.region_id.astype(str)
metro_condo = metro_condo.merge(major_metros, on='region_id', how='inner')
metro_condo

C:\Users\Brian\AppData\Roaming\Python\Python37\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,indicator_id,region_id,date,value,region
0,ZCON,394913,2020-06-30,498772.0,"New York, NY"
1,ZCON,394913,2020-04-30,503867.0,"New York, NY"
2,ZCON,394913,2020-03-31,503997.0,"New York, NY"
3,ZCON,394913,2020-10-31,498499.0,"New York, NY"
4,ZCON,394913,2020-12-31,498221.0,"New York, NY"
...,...,...,...,...,...
9103,ZCON,394561,2019-10-31,110684.0,"El Paso, TX"
9104,ZCON,394561,2019-11-30,110444.0,"El Paso, TX"
9105,ZCON,394561,2019-12-31,110818.0,"El Paso, TX"
9106,ZCON,394561,2020-01-31,111086.0,"El Paso, TX"


In [170]:
aus_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[0]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
aus_condo_df = pd.concat([aus_condo_df, temp_df])
aus_condo_df.to_csv('./Resources/aus_condo.csv', index=False)
aus_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
6668,ZCON,394355,2020-06-30,327426.0,"Austin, TX",NaN,NaN
6669,ZCON,394355,2020-04-30,320088.0,"Austin, TX",-2.241117,NaN
6670,ZCON,394355,2020-03-31,318084.0,"Austin, TX",-0.626078,-2.241117
6671,ZCON,394355,2020-10-31,329129.0,"Austin, TX",3.472353,-0.626078
6672,ZCON,394355,2020-12-31,354331.0,"Austin, TX",7.657180,3.472353
...,...,...,...,...,...,...,...
6968,ZCON,394355,2019-10-31,314073.0,"Austin, TX",0.126883,0.395597
6969,ZCON,394355,2019-11-30,315075.0,"Austin, TX",0.319034,0.126883
6970,ZCON,394355,2019-12-31,315778.0,"Austin, TX",0.223121,0.319034
6971,ZCON,394355,2020-01-31,316926.0,"Austin, TX",0.363547,0.223121


In [171]:
bal_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[1]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
bal_condo_df = pd.concat([bal_condo_df, temp_df])
bal_condo_df.to_csv('./Resources/bal_condo.csv', index=False)
bal_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
3965,ZCON,394358,2020-06-30,212796.0,"Baltimore, MD",NaN,NaN
3966,ZCON,394358,2020-04-30,212613.0,"Baltimore, MD",-0.085998,NaN
3967,ZCON,394358,2020-03-31,212303.0,"Baltimore, MD",-0.145805,-0.085998
3968,ZCON,394358,2020-10-31,213955.0,"Baltimore, MD",0.778133,-0.145805
3969,ZCON,394358,2020-12-31,220302.0,"Baltimore, MD",2.966512,0.778133
...,...,...,...,...,...,...,...
4265,ZCON,394358,2019-10-31,210012.0,"Baltimore, MD",0.144963,0.157131
4266,ZCON,394358,2019-11-30,210229.0,"Baltimore, MD",0.103327,0.144963
4267,ZCON,394358,2019-12-31,210646.0,"Baltimore, MD",0.198355,0.103327
4268,ZCON,394358,2020-01-31,211068.0,"Baltimore, MD",0.200336,0.198355


In [172]:
bos_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[2]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
bos_condo_df = pd.concat([bos_condo_df, temp_df])
bos_condo_df.to_csv('./Resources/bos_condo.csv', index=False)
bos_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
2135,ZCON,394404,2020-06-30,439638.0,"Boston, MA",NaN,NaN
2136,ZCON,394404,2020-04-30,437341.0,"Boston, MA",-0.522475,NaN
2137,ZCON,394404,2020-03-31,435865.0,"Boston, MA",-0.337494,-0.522475
2138,ZCON,394404,2020-10-31,446149.0,"Boston, MA",2.359446,-0.337494
2139,ZCON,394404,2020-12-31,451657.0,"Boston, MA",1.234565,2.359446
...,...,...,...,...,...,...,...
2435,ZCON,394404,2019-10-31,431153.0,"Boston, MA",0.177048,0.227519
2436,ZCON,394404,2019-11-30,431692.0,"Boston, MA",0.125014,0.177048
2437,ZCON,394404,2019-12-31,432233.0,"Boston, MA",0.125321,0.125014
2438,ZCON,394404,2020-01-31,433194.0,"Boston, MA",0.222334,0.125321


In [173]:
char_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[3]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
char_condo_df = pd.concat([char_condo_df, temp_df])
char_condo_df.to_csv('./Resources/char_condo.csv', index=False)
char_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
4880,ZCON,394458,2020-06-30,194726.0,"Charlotte, NC",NaN,NaN
4881,ZCON,394458,2020-04-30,191972.0,"Charlotte, NC",-1.414295,NaN
4882,ZCON,394458,2020-03-31,190597.0,"Charlotte, NC",-0.716250,-1.414295
4883,ZCON,394458,2020-10-31,207188.0,"Charlotte, NC",8.704754,-0.716250
4884,ZCON,394458,2020-12-31,211721.0,"Charlotte, NC",2.187868,8.704754
...,...,...,...,...,...,...,...
5180,ZCON,394458,2019-10-31,184987.0,"Charlotte, NC",0.553901,0.402225
5181,ZCON,394458,2019-11-30,185983.0,"Charlotte, NC",0.538416,0.553901
5182,ZCON,394458,2019-12-31,186941.0,"Charlotte, NC",0.515101,0.538416
5183,ZCON,394458,2020-01-31,187559.0,"Charlotte, NC",0.330586,0.515101


In [174]:
chi_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[4]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
chi_condo_df = pd.concat([chi_condo_df, temp_df])
chi_condo_df.to_csv('./Resources/chi_condo.csv', index=False)
chi_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
610,ZCON,394463,2020-06-30,206792.0,"Chicago, IL",NaN,NaN
611,ZCON,394463,2020-04-30,206798.0,"Chicago, IL",0.002901,NaN
612,ZCON,394463,2020-03-31,206579.0,"Chicago, IL",-0.105900,0.002901
613,ZCON,394463,2020-10-31,208817.0,"Chicago, IL",1.083363,-0.105900
614,ZCON,394463,2020-11-30,209581.0,"Chicago, IL",0.365871,1.083363
...,...,...,...,...,...,...,...
910,ZCON,394463,2019-10-31,205687.0,"Chicago, IL",0.030152,0.044275
911,ZCON,394463,2019-11-30,205712.0,"Chicago, IL",0.012154,0.030152
912,ZCON,394463,2019-12-31,205782.0,"Chicago, IL",0.034028,0.012154
913,ZCON,394463,2020-01-31,206002.0,"Chicago, IL",0.106909,0.034028


In [175]:
col_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[5]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
col_condo_df = pd.concat([col_condo_df, temp_df])
col_condo_df.to_csv('./Resources/col_condo.csv', index=False)
col_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
5795,ZCON,394492,2020-06-30,162700.0,"Columbus, OH",NaN,NaN
5796,ZCON,394492,2020-04-30,161782.0,"Columbus, OH",-0.564229,NaN
5797,ZCON,394492,2020-03-31,160936.0,"Columbus, OH",-0.522926,-0.564229
5798,ZCON,394492,2020-10-31,167387.0,"Columbus, OH",4.008426,-0.522926
5799,ZCON,394492,2020-12-31,172124.0,"Columbus, OH",2.829969,4.008426
...,...,...,...,...,...,...,...
6095,ZCON,394492,2019-10-31,156320.0,"Columbus, OH",0.514403,0.591180
6096,ZCON,394492,2019-11-30,157141.0,"Columbus, OH",0.525205,0.514403
6097,ZCON,394492,2019-12-31,158114.0,"Columbus, OH",0.619189,0.525205
6098,ZCON,394492,2020-01-31,159352.0,"Columbus, OH",0.782979,0.619189


In [176]:
dal_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[6]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
dal_condo_df = pd.concat([dal_condo_df, temp_df])
dal_condo_df.to_csv('./Resources/dal_condo.csv', index=False)
dal_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
915,ZCON,394514,2020-06-30,184800.0,"Dallas-Fort Worth, TX",NaN,NaN
916,ZCON,394514,2020-04-30,181579.0,"Dallas-Fort Worth, TX",-1.742965,NaN
917,ZCON,394514,2020-03-31,180977.0,"Dallas-Fort Worth, TX",-0.331536,-1.742965
918,ZCON,394514,2020-10-31,188504.0,"Dallas-Fort Worth, TX",4.159092,-0.331536
919,ZCON,394514,2020-12-31,194766.0,"Dallas-Fort Worth, TX",3.321945,4.159092
...,...,...,...,...,...,...,...
1215,ZCON,394514,2019-10-31,180375.0,"Dallas-Fort Worth, TX",0.408593,0.890736
1216,ZCON,394514,2019-11-30,181048.0,"Dallas-Fort Worth, TX",0.373112,0.408593
1217,ZCON,394514,2019-12-31,180681.0,"Dallas-Fort Worth, TX",-0.202709,0.373112
1218,ZCON,394514,2020-01-31,180809.0,"Dallas-Fort Worth, TX",0.070843,-0.202709


In [177]:
den_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[7]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
den_condo_df = pd.concat([den_condo_df, temp_df])
den_condo_df.to_csv('./Resources/den_condo.csv', index=False)
den_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
4270,ZCON,394530,2020-06-30,282247.0,"Denver, CO",NaN,NaN
4271,ZCON,394530,2020-04-30,280927.0,"Denver, CO",-0.467675,NaN
4272,ZCON,394530,2020-03-31,280224.0,"Denver, CO",-0.250243,-0.467675
4273,ZCON,394530,2020-10-31,289367.0,"Denver, CO",3.262747,-0.250243
4274,ZCON,394530,2020-12-31,295022.0,"Denver, CO",1.954266,3.262747
...,...,...,...,...,...,...,...
4570,ZCON,394530,2019-10-31,278012.0,"Denver, CO",0.102979,0.123295
4571,ZCON,394530,2019-11-30,278310.0,"Denver, CO",0.107190,0.102979
4572,ZCON,394530,2019-12-31,278961.0,"Denver, CO",0.233912,0.107190
4573,ZCON,394530,2020-01-31,279992.0,"Denver, CO",0.369586,0.233912


In [178]:
det_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[8]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
det_condo_df = pd.concat([det_condo_df, temp_df])
det_condo_df.to_csv('./Resources/det_condo.csv', index=False)
det_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
2745,ZCON,394532,2020-06-30,204689.0,"Detroit, MI",NaN,NaN
2746,ZCON,394532,2020-04-30,205932.0,"Detroit, MI",0.607263,NaN
2747,ZCON,394532,2020-03-31,205080.0,"Detroit, MI",-0.413729,0.607263
2748,ZCON,394532,2020-10-31,198120.0,"Detroit, MI",-3.393798,-0.413729
2749,ZCON,394532,2020-12-31,203386.0,"Detroit, MI",2.657985,-3.393798
...,...,...,...,...,...,...,...
3045,ZCON,394532,2019-10-31,203711.0,"Detroit, MI",0.190337,0.225271
3046,ZCON,394532,2019-11-30,203945.0,"Detroit, MI",0.114869,0.190337
3047,ZCON,394532,2019-12-31,204198.0,"Detroit, MI",0.124053,0.114869
3048,ZCON,394532,2020-01-31,204463.0,"Detroit, MI",0.129776,0.124053


In [179]:
elp_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[9]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
elp_condo_df = pd.concat([elp_condo_df, temp_df])
elp_condo_df.to_csv('./Resources/elp_condo.csv', index=False)
elp_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
8803,ZCON,394561,2020-06-30,110751.0,"El Paso, TX",NaN,NaN
8804,ZCON,394561,2020-04-30,111248.0,"El Paso, TX",0.448754,NaN
8805,ZCON,394561,2020-03-31,111304.0,"El Paso, TX",0.050338,0.448754
8806,ZCON,394561,2020-10-31,109320.0,"El Paso, TX",-1.782506,0.050338
8807,ZCON,394561,2020-12-31,113146.0,"El Paso, TX",3.499817,-1.782506
...,...,...,...,...,...,...,...
9103,ZCON,394561,2019-10-31,110684.0,"El Paso, TX",-0.280193,-0.288368
9104,ZCON,394561,2019-11-30,110444.0,"El Paso, TX",-0.216834,-0.280193
9105,ZCON,394561,2019-12-31,110818.0,"El Paso, TX",0.338633,-0.216834
9106,ZCON,394561,2020-01-31,111086.0,"El Paso, TX",0.241838,0.338633


In [180]:
hou_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[10]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
hou_condo_df = pd.concat([hou_condo_df, temp_df])
hou_condo_df.to_csv('./Resources/hou_condo.csv', index=False)
hou_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
1525,ZCON,394692,2020-06-30,135571.0,"Houston, TX",NaN,NaN
1526,ZCON,394692,2020-04-30,132738.0,"Houston, TX",-2.089680,NaN
1527,ZCON,394692,2020-03-31,132334.0,"Houston, TX",-0.304359,-2.089680
1528,ZCON,394692,2020-10-31,145306.0,"Houston, TX",9.802470,-0.304359
1529,ZCON,394692,2020-12-31,147352.0,"Houston, TX",1.408063,9.802470
...,...,...,...,...,...,...,...
1825,ZCON,394692,2019-10-31,132543.0,"Houston, TX",0.211699,0.081722
1826,ZCON,394692,2019-11-30,132746.0,"Houston, TX",0.153158,0.211699
1827,ZCON,394692,2019-12-31,132948.0,"Houston, TX",0.152170,0.153158
1828,ZCON,394692,2020-01-31,132942.0,"Houston, TX",-0.004513,0.152170


In [181]:
ind_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[11]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
ind_condo_df = pd.concat([ind_condo_df, temp_df])
ind_condo_df.to_csv('./Resources/ind_condo.csv', index=False)
ind_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
6100,ZCON,394705,2020-06-30,158255.0,"Indianapolis, IN",NaN,NaN
6101,ZCON,394705,2020-04-30,156490.0,"Indianapolis, IN",-1.115289,NaN
6102,ZCON,394705,2020-03-31,155480.0,"Indianapolis, IN",-0.645409,-1.115289
6103,ZCON,394705,2020-10-31,164081.0,"Indianapolis, IN",5.531901,-0.645409
6104,ZCON,394705,2020-12-31,165265.0,"Indianapolis, IN",0.721595,5.531901
...,...,...,...,...,...,...,...
6358,ZCON,394705,2019-10-31,151578.0,"Indianapolis, IN",0.612658,0.607700
6359,ZCON,394705,2019-11-30,152379.0,"Indianapolis, IN",0.528441,0.612658
6360,ZCON,394705,2019-12-31,152872.0,"Indianapolis, IN",0.323535,0.528441
6361,ZCON,394705,2020-01-31,153477.0,"Indianapolis, IN",0.395756,0.323535


In [182]:
jak_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[12]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
jak_condo_df = pd.concat([jak_condo_df, temp_df])
jak_condo_df.to_csv('./Resources/jak_condo.csv', index=False)
jak_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
7583,ZCON,394714,2020-06-30,178911.0,"Jacksonville, FL",NaN,NaN
7584,ZCON,394714,2020-04-30,178621.0,"Jacksonville, FL",-0.162092,NaN
7585,ZCON,394714,2020-03-31,178075.0,"Jacksonville, FL",-0.305675,-0.162092
7586,ZCON,394714,2020-10-31,180040.0,"Jacksonville, FL",1.103468,-0.305675
7587,ZCON,394714,2020-12-31,185149.0,"Jacksonville, FL",2.837703,1.103468
...,...,...,...,...,...,...,...
7883,ZCON,394714,2019-10-31,173518.0,"Jacksonville, FL",0.277396,0.346205
7884,ZCON,394714,2019-11-30,173907.0,"Jacksonville, FL",0.224184,0.277396
7885,ZCON,394714,2019-12-31,174419.0,"Jacksonville, FL",0.294410,0.224184
7886,ZCON,394714,2020-01-31,175238.0,"Jacksonville, FL",0.469559,0.294410


In [183]:
lvn_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[13]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
lvn_condo_df = pd.concat([lvn_condo_df, temp_df])
lvn_condo_df.to_csv('./Resources/lvn_condo.csv', index=False)
lvn_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
5490,ZCON,394775,2020-06-30,163007.0,"Las Vegas, NV",NaN,NaN
5491,ZCON,394775,2020-04-30,162607.0,"Las Vegas, NV",-0.245388,NaN
5492,ZCON,394775,2020-03-31,162044.0,"Las Vegas, NV",-0.346234,-0.245388
5493,ZCON,394775,2020-10-31,166607.0,"Las Vegas, NV",2.815902,-0.346234
5494,ZCON,394775,2020-12-31,169355.0,"Las Vegas, NV",1.649390,2.815902
...,...,...,...,...,...,...,...
5790,ZCON,394775,2019-10-31,160403.0,"Las Vegas, NV",0.121717,-0.104131
5791,ZCON,394775,2019-11-30,160873.0,"Las Vegas, NV",0.293012,0.121717
5792,ZCON,394775,2019-12-31,161145.0,"Las Vegas, NV",0.169077,0.293012
5793,ZCON,394775,2020-01-31,161564.0,"Las Vegas, NV",0.260014,0.169077


In [184]:
lac_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[14]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
lac_condo_df = pd.concat([lac_condo_df, temp_df])
lac_condo_df.to_csv('./Resources/lac_condo.csv', index=False)
lac_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
305,ZCON,753899,2020-06-30,547500.0,"Los Angeles-Long Beach-Anaheim, CA",NaN,NaN
306,ZCON,753899,2020-04-30,544521.0,"Los Angeles-Long Beach-Anaheim, CA",-0.544110,NaN
307,ZCON,753899,2020-03-31,541888.0,"Los Angeles-Long Beach-Anaheim, CA",-0.483544,-0.544110
308,ZCON,753899,2020-10-31,565625.0,"Los Angeles-Long Beach-Anaheim, CA",4.380425,-0.483544
309,ZCON,753899,2020-12-31,579381.0,"Los Angeles-Long Beach-Anaheim, CA",2.432000,4.380425
...,...,...,...,...,...,...,...
605,ZCON,753899,2019-10-31,538230.0,"Los Angeles-Long Beach-Anaheim, CA",0.277789,0.381523
606,ZCON,753899,2019-11-30,539075.0,"Los Angeles-Long Beach-Anaheim, CA",0.156996,0.277789
607,ZCON,753899,2019-12-31,540508.0,"Los Angeles-Long Beach-Anaheim, CA",0.265826,0.156996
608,ZCON,753899,2020-01-31,542310.0,"Los Angeles-Long Beach-Anaheim, CA",0.333390,0.265826


In [185]:
lou_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[15]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
lou_condo_df = pd.concat([lou_condo_df, temp_df])
lou_condo_df.to_csv('./Resources/lou_condo.csv', index=False)
lou_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
8498,ZCON,394807,2020-06-30,156001.0,"Louisville-Jefferson County, KY",NaN,NaN
8499,ZCON,394807,2020-04-30,155030.0,"Louisville-Jefferson County, KY",-0.622432,NaN
8500,ZCON,394807,2020-03-31,154256.0,"Louisville-Jefferson County, KY",-0.499258,-0.622432
8501,ZCON,394807,2020-10-31,160636.0,"Louisville-Jefferson County, KY",4.135982,-0.499258
8502,ZCON,394807,2020-12-31,162576.0,"Louisville-Jefferson County, KY",1.207699,4.135982
...,...,...,...,...,...,...,...
8798,ZCON,394807,2019-10-31,151472.0,"Louisville-Jefferson County, KY",0.312583,0.404277
8799,ZCON,394807,2019-11-30,152135.0,"Louisville-Jefferson County, KY",0.437705,0.312583
8800,ZCON,394807,2019-12-31,152782.0,"Louisville-Jefferson County, KY",0.425280,0.437705
8801,ZCON,394807,2020-01-31,153568.0,"Louisville-Jefferson County, KY",0.514459,0.425280


In [186]:
mem_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[16]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
mem_condo_df = pd.concat([mem_condo_df, temp_df])
mem_condo_df.to_csv('./Resources/mem_condo.csv', index=False)
mem_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
7888,ZCON,394849,2020-06-30,206358.0,"Memphis, TN",NaN,NaN
7889,ZCON,394849,2020-04-30,205242.0,"Memphis, TN",-0.540808,NaN
7890,ZCON,394849,2020-03-31,203762.0,"Memphis, TN",-0.721100,-0.540808
7891,ZCON,394849,2020-10-31,188093.0,"Memphis, TN",-7.689854,-0.721100
7892,ZCON,394849,2020-12-31,191566.0,"Memphis, TN",1.846427,-7.689854
...,...,...,...,...,...,...,...
8188,ZCON,394849,2019-10-31,198528.0,"Memphis, TN",0.540360,0.561729
8189,ZCON,394849,2019-11-30,199817.0,"Memphis, TN",0.649279,0.540360
8190,ZCON,394849,2019-12-31,201228.0,"Memphis, TN",0.706146,0.649279
8191,ZCON,394849,2020-01-31,202166.0,"Memphis, TN",0.466138,0.706146


In [187]:
mil_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[17]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
mil_condo_df = pd.concat([mil_condo_df, temp_df])
mil_condo_df.to_csv('./Resources/mil_condo.csv', index=False)
mil_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
7278,ZCON,394862,2020-06-30,183791.0,"Milwaukee, WI",NaN,NaN
7279,ZCON,394862,2020-04-30,182675.0,"Milwaukee, WI",-0.607211,NaN
7280,ZCON,394862,2020-03-31,181874.0,"Milwaukee, WI",-0.438484,-0.607211
7281,ZCON,394862,2020-10-31,185725.0,"Milwaukee, WI",2.117400,-0.438484
7282,ZCON,394862,2020-12-31,191777.0,"Milwaukee, WI",3.258581,2.117400
...,...,...,...,...,...,...,...
7578,ZCON,394862,2019-10-31,177377.0,"Milwaukee, WI",0.441115,0.384261
7579,ZCON,394862,2019-11-30,178246.0,"Milwaukee, WI",0.489917,0.441115
7580,ZCON,394862,2019-12-31,179244.0,"Milwaukee, WI",0.559900,0.489917
7581,ZCON,394862,2020-01-31,180269.0,"Milwaukee, WI",0.571846,0.559900


In [188]:
nash_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[18]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
nash_condo_df = pd.concat([nash_condo_df, temp_df])
nash_condo_df.to_csv('./Resources/nash_condo.csv', index=False)
nash_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
6973,ZCON,394902,2020-06-30,256833.0,"Nashville, TN",NaN,NaN
6974,ZCON,394902,2020-04-30,252197.0,"Nashville, TN",-1.805064,NaN
6975,ZCON,394902,2020-03-31,250530.0,"Nashville, TN",-0.660991,-1.805064
6976,ZCON,394902,2020-10-31,259186.0,"Nashville, TN",3.455075,-0.660991
6977,ZCON,394902,2020-12-31,263006.0,"Nashville, TN",1.473845,3.455075
...,...,...,...,...,...,...,...
7273,ZCON,394902,2019-10-31,245827.0,"Nashville, TN",0.265932,0.197391
7274,ZCON,394902,2019-11-30,246763.0,"Nashville, TN",0.380756,0.265932
7275,ZCON,394902,2019-12-31,247696.0,"Nashville, TN",0.378096,0.380756
7276,ZCON,394902,2020-01-31,248755.0,"Nashville, TN",0.427540,0.378096


In [189]:
nyc_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[19]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
nyc_condo_df = pd.concat([nyc_condo_df, temp_df])
nyc_condo_df.to_csv('./Resources/nyc_condo.csv', index=False)
nyc_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
0,ZCON,394913,2020-06-30,498772.0,"New York, NY",NaN,NaN
1,ZCON,394913,2020-04-30,503867.0,"New York, NY",1.021509,NaN
2,ZCON,394913,2020-03-31,503997.0,"New York, NY",0.025800,1.021509
3,ZCON,394913,2020-10-31,498499.0,"New York, NY",-1.090880,0.025800
4,ZCON,394913,2020-12-31,498221.0,"New York, NY",-0.055767,-1.090880
...,...,...,...,...,...,...,...
300,ZCON,394913,2019-10-31,504294.0,"New York, NY",-0.192769,-0.174454
301,ZCON,394913,2019-11-30,503516.0,"New York, NY",-0.154275,-0.192769
302,ZCON,394913,2019-12-31,503570.0,"New York, NY",0.010725,-0.154275
303,ZCON,394913,2020-01-31,503870.0,"New York, NY",0.059575,0.010725


In [190]:
okc_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[20]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
okc_condo_df = pd.concat([okc_condo_df, temp_df])
okc_condo_df.to_csv('./Resources/okc_condo.csv', index=False)
okc_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
8193,ZCON,394935,2020-06-30,92089.0,"Oklahoma City, OK",NaN,NaN
8194,ZCON,394935,2020-04-30,89883.0,"Oklahoma City, OK",-2.395509,NaN
8195,ZCON,394935,2020-03-31,89526.0,"Oklahoma City, OK",-0.397183,-2.395509
8196,ZCON,394935,2020-12-31,91331.0,"Oklahoma City, OK",2.016174,-0.397183
8197,ZCON,394935,2020-08-31,93017.0,"Oklahoma City, OK",1.846033,2.016174
...,...,...,...,...,...,...,...
8493,ZCON,394935,2019-10-31,88038.0,"Oklahoma City, OK",0.480500,0.655975
8494,ZCON,394935,2019-11-30,88289.0,"Oklahoma City, OK",0.285104,0.480500
8495,ZCON,394935,2019-12-31,88425.0,"Oklahoma City, OK",0.154040,0.285104
8496,ZCON,394935,2020-01-31,88825.0,"Oklahoma City, OK",0.452361,0.154040


In [191]:
phi_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[21]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
phi_condo_df = pd.concat([phi_condo_df, temp_df])
phi_condo_df.to_csv('./Resources/phi_condo.csv', index=False)
phi_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
1220,ZCON,394974,2020-06-30,203776.0,"Philadelphia, PA",NaN,NaN
1221,ZCON,394974,2020-04-30,201048.0,"Philadelphia, PA",-1.338725,NaN
1222,ZCON,394974,2020-03-31,200308.0,"Philadelphia, PA",-0.368071,-1.338725
1223,ZCON,394974,2020-10-31,212892.0,"Philadelphia, PA",6.282325,-0.368071
1224,ZCON,394974,2020-12-31,216595.0,"Philadelphia, PA",1.739380,6.282325
...,...,...,...,...,...,...,...
1520,ZCON,394974,2019-10-31,197908.0,"Philadelphia, PA",0.219776,0.252314
1521,ZCON,394974,2019-11-30,198286.0,"Philadelphia, PA",0.190998,0.219776
1522,ZCON,394974,2019-12-31,198748.0,"Philadelphia, PA",0.232997,0.190998
1523,ZCON,394974,2020-01-31,199253.0,"Philadelphia, PA",0.254091,0.232997


In [192]:
phx_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[22]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
phx_condo_df = pd.concat([phx_condo_df, temp_df])
phx_condo_df.to_csv('./Resources/phx_condo.csv', index=False)
phx_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
3050,ZCON,394976,2020-06-30,202809.0,"Phoenix, AZ",NaN,NaN
3051,ZCON,394976,2020-04-30,197343.0,"Phoenix, AZ",-2.695147,NaN
3052,ZCON,394976,2020-03-31,195026.0,"Phoenix, AZ",-1.174098,-2.695147
3053,ZCON,394976,2020-10-31,210020.0,"Phoenix, AZ",7.688206,-1.174098
3054,ZCON,394976,2020-12-31,216920.0,"Phoenix, AZ",3.285401,7.688206
...,...,...,...,...,...,...,...
3350,ZCON,394976,2019-10-31,186751.0,"Phoenix, AZ",0.735758,0.625288
3351,ZCON,394976,2019-11-30,188160.0,"Phoenix, AZ",0.754481,0.735758
3352,ZCON,394976,2019-12-31,189782.0,"Phoenix, AZ",0.862032,0.754481
3353,ZCON,394976,2020-01-31,191312.0,"Phoenix, AZ",0.806188,0.862032


In [193]:
port_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[23]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
port_condo_df = pd.concat([port_condo_df, temp_df])
port_condo_df.to_csv('./Resources/port_condo.csv', index=False)
port_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
4575,ZCON,394998,2020-06-30,303733.0,"Portland, OR",NaN,NaN
4576,ZCON,394998,2020-04-30,302761.0,"Portland, OR",-0.320018,NaN
4577,ZCON,394998,2020-03-31,301993.0,"Portland, OR",-0.253665,-0.320018
4578,ZCON,394998,2020-10-31,307107.0,"Portland, OR",1.693417,-0.253665
4579,ZCON,394998,2020-12-31,318395.0,"Portland, OR",3.675592,1.693417
...,...,...,...,...,...,...,...
4875,ZCON,394998,2019-10-31,300558.0,"Portland, OR",0.225088,0.111835
4876,ZCON,394998,2019-11-30,301006.0,"Portland, OR",0.149056,0.225088
4877,ZCON,394998,2019-12-31,301108.0,"Portland, OR",0.033886,0.149056
4878,ZCON,394998,2020-01-31,300988.0,"Portland, OR",-0.039853,0.033886


In [194]:
san_ant_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[24]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
san_ant_condo_df = pd.concat([san_ant_condo_df, temp_df])
san_ant_condo_df.to_csv('./Resources/san_ant_condo.csv', index=False)
san_ant_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
5185,ZCON,395055,2020-06-30,152199.0,"San Antonio, TX",NaN,NaN
5186,ZCON,395055,2020-04-30,150982.0,"San Antonio, TX",-0.799611,NaN
5187,ZCON,395055,2020-03-31,150880.0,"San Antonio, TX",-0.067558,-0.799611
5188,ZCON,395055,2020-10-31,153900.0,"San Antonio, TX",2.001591,-0.067558
5189,ZCON,395055,2020-12-31,158372.0,"San Antonio, TX",2.905783,2.001591
...,...,...,...,...,...,...,...
5485,ZCON,395055,2019-10-31,149940.0,"San Antonio, TX",0.175042,0.361408
5486,ZCON,395055,2019-11-30,150300.0,"San Antonio, TX",0.240096,0.175042
5487,ZCON,395055,2019-12-31,150199.0,"San Antonio, TX",-0.067199,0.240096
5488,ZCON,395055,2020-01-31,150442.0,"San Antonio, TX",0.161785,-0.067199


In [195]:
sand_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[25]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
sand_condo_df = pd.concat([sand_condo_df, temp_df])
sand_condo_df.to_csv('./Resources/sand_condo.csv', index=False)
sand_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
3660,ZCON,395056,2020-06-30,459405.0,"San Diego, CA",NaN,NaN
3661,ZCON,395056,2020-04-30,459486.0,"San Diego, CA",0.017632,NaN
3662,ZCON,395056,2020-03-31,458431.0,"San Diego, CA",-0.229604,0.017632
3663,ZCON,395056,2020-10-31,475158.0,"San Diego, CA",3.648750,-0.229604
3664,ZCON,395056,2020-12-31,489612.0,"San Diego, CA",3.041936,3.648750
...,...,...,...,...,...,...,...
3960,ZCON,395056,2019-10-31,446723.0,"San Diego, CA",0.219183,0.225298
3961,ZCON,395056,2019-11-30,448469.0,"San Diego, CA",0.390846,0.219183
3962,ZCON,395056,2019-12-31,450468.0,"San Diego, CA",0.445739,0.390846
3963,ZCON,395056,2020-01-31,453089.0,"San Diego, CA",0.581839,0.445739


In [196]:
sanf_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[26]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
sanf_condo_df = pd.concat([sanf_condo_df, temp_df])
sanf_condo_df.to_csv('./Resources/sanf_condo.csv', index=False)
sanf_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
2440,ZCON,395057,2020-06-30,867623.0,"San Francisco, CA",NaN,NaN
2441,ZCON,395057,2020-04-30,883723.0,"San Francisco, CA",1.855645,NaN
2442,ZCON,395057,2020-03-31,885285.0,"San Francisco, CA",0.176752,1.855645
2443,ZCON,395057,2020-10-31,853486.0,"San Francisco, CA",-3.591951,0.176752
2444,ZCON,395057,2020-12-31,860706.0,"San Francisco, CA",0.845942,-3.591951
...,...,...,...,...,...,...,...
2740,ZCON,395057,2019-10-31,880868.0,"San Francisco, CA",0.130838,0.116992
2741,ZCON,395057,2019-11-30,882516.0,"San Francisco, CA",0.187088,0.130838
2742,ZCON,395057,2019-12-31,886619.0,"San Francisco, CA",0.464921,0.187088
2743,ZCON,395057,2020-01-31,889916.0,"San Francisco, CA",0.371862,0.464921


In [197]:
san_jo_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[27]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
san_jo_condo_df = pd.concat([san_jo_condo_df, temp_df])
san_jo_condo_df.to_csv('./Resources/san_jo_condo.csv', index=False)
san_jo_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
6363,ZCON,395059,2020-06-30,796821.0,"San Jose, CA",NaN,NaN
6364,ZCON,395059,2020-04-30,802274.0,"San Jose, CA",0.684344,NaN
6365,ZCON,395059,2020-03-31,800523.0,"San Jose, CA",-0.218255,0.684344
6366,ZCON,395059,2020-10-31,805993.0,"San Jose, CA",0.683303,-0.218255
6367,ZCON,395059,2020-12-31,838781.0,"San Jose, CA",4.068025,0.683303
...,...,...,...,...,...,...,...
6663,ZCON,395059,2019-10-31,774319.0,"San Jose, CA",0.186187,0.192768
6664,ZCON,395059,2019-11-30,776796.0,"San Jose, CA",0.319894,0.186187
6665,ZCON,395059,2019-12-31,780582.0,"San Jose, CA",0.487387,0.319894
6666,ZCON,395059,2020-01-31,787915.0,"San Jose, CA",0.939427,0.487387


In [198]:
sea_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[28]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
sea_condo_df = pd.concat([sea_condo_df, temp_df])
sea_condo_df.to_csv('./Resources/sea_condo.csv', index=False)
sea_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
3355,ZCON,395078,2020-06-30,402239.0,"Seattle, WA",NaN,NaN
3356,ZCON,395078,2020-04-30,398159.0,"Seattle, WA",-1.014322,NaN
3357,ZCON,395078,2020-03-31,395487.0,"Seattle, WA",-0.671089,-1.014322
3358,ZCON,395078,2020-10-31,418429.0,"Seattle, WA",5.800949,-0.671089
3359,ZCON,395078,2020-12-31,426892.0,"Seattle, WA",2.022565,5.800949
...,...,...,...,...,...,...,...
3655,ZCON,395078,2019-10-31,383405.0,"Seattle, WA",0.198356,0.185895
3656,ZCON,395078,2019-11-30,385094.0,"Seattle, WA",0.440526,0.198356
3657,ZCON,395078,2019-12-31,387834.0,"Seattle, WA",0.711515,0.440526
3658,ZCON,395078,2020-01-31,390847.0,"Seattle, WA",0.776879,0.711515


In [199]:
dc_condo_df = pd.DataFrame()
temp_df = metro_condo[metro_condo.region == majors_list[29]]
temp_df['change'] = temp_df['value'].pct_change() * 100
temp_df['lagged_change'] = temp_df['change'].shift()
dc_condo_df = pd.concat([dc_condo_df, temp_df])
dc_condo_df.to_csv('./Resources/dc_condo.csv', index=False)
dc_condo_df

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,indicator_id,region_id,date,value,region,change,lagged_change
1830,ZCON,395209,2020-06-30,315319.0,"Washington, DC",NaN,NaN
1831,ZCON,395209,2020-04-30,310743.0,"Washington, DC",-1.451229,NaN
1832,ZCON,395209,2020-03-31,309277.0,"Washington, DC",-0.471772,-1.451229
1833,ZCON,395209,2020-10-31,316116.0,"Washington, DC",2.211286,-0.471772
1834,ZCON,395209,2020-12-31,324293.0,"Washington, DC",2.586709,2.211286
...,...,...,...,...,...,...,...
2130,ZCON,395209,2019-10-31,304686.0,"Washington, DC",0.474862,0.439855
2131,ZCON,395209,2019-11-30,306204.0,"Washington, DC",0.498218,0.474862
2132,ZCON,395209,2019-12-31,307492.0,"Washington, DC",0.420635,0.498218
2133,ZCON,395209,2020-01-31,308658.0,"Washington, DC",0.379197,0.420635


In [200]:
# Reset the index for the new dataframe before writing to csv
metro_homes = metro_homes.reset_index(drop=True)
metro_homes.sort_values(by='date', inplace=True)
metro_homes

,indicator_id,region_id,date,value
534143,Z2BR,394314,1996-01-31,28575.0
764998,Z3BR,394949,1996-01-31,73795.0
480862,Z2BR,394308,1996-01-31,86397.0
562650,Z2BR,394501,1996-01-31,46682.0
843860,Z3BR,395178,1996-01-31,84852.0
...,...,...,...,...
59337,ZCON,394447,2021-05-31,117956.0
70443,Z1BR,394734,2021-05-31,82249.0
59334,ZCON,394471,2021-05-31,223577.0
59349,ZCON,394645,2021-05-31,414356.0


In [201]:
metro_homes.to_csv('./Resources/metro_homes.csv', index=False,)
metro_homes

,indicator_id,region_id,date,value
534143,Z2BR,394314,1996-01-31,28575.0
764998,Z3BR,394949,1996-01-31,73795.0
480862,Z2BR,394308,1996-01-31,86397.0
562650,Z2BR,394501,1996-01-31,46682.0
843860,Z3BR,395178,1996-01-31,84852.0
...,...,...,...,...
59337,ZCON,394447,2021-05-31,117956.0
70443,Z1BR,394734,2021-05-31,82249.0
59334,ZCON,394471,2021-05-31,223577.0
59349,ZCON,394645,2021-05-31,414356.0


In [202]:
metro_condo = metro_homes[metro_homes.indicator_id == 'ZCON']
metro_condo.sort_values(by='date', inplace=True)
metro_condo

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,indicator_id,region_id,date,value
96350,ZCON,394619,1996-01-31,70863.0
149890,ZCON,394314,1996-01-31,85074.0
106031,ZCON,394528,1996-01-31,105987.0
195349,ZCON,394764,1996-01-31,49046.0
81023,ZCON,394463,1996-01-31,135785.0
...,...,...,...,...
59964,ZCON,394580,2021-05-31,147685.0
59925,ZCON,394709,2021-05-31,190761.0
59922,ZCON,394599,2021-05-31,180417.0
60021,ZCON,394907,2021-05-31,196293.0


In [204]:
metro_1br = metro_homes[metro_homes.indicator_id == 'Z1BR']
metro_1br.sort_values(by='date', inplace=True)
metro_1br

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,indicator_id,region_id,date,value
1042909,Z1BR,394891,1996-01-31,42905.0
1061713,Z1BR,753892,1996-01-31,26374.0
978424,Z1BR,394788,1996-01-31,87144.0
1007727,Z1BR,394829,1996-01-31,60190.0
889453,Z1BR,394463,1996-01-31,99397.0
...,...,...,...,...
71740,Z1BR,394886,2021-05-31,105399.0
71743,Z1BR,394962,2021-05-31,57716.0
71746,Z1BR,394491,2021-05-31,96095.0
70065,Z1BR,394346,2021-05-31,171744.0


In [205]:
metro_2br = metro_homes[metro_homes.indicator_id == 'Z2BR']
metro_2br.sort_values(by='date', inplace=True)
metro_2br

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,indicator_id,region_id,date,value
534143,Z2BR,394314,1996-01-31,28575.0
508618,Z2BR,394808,1996-01-31,40731.0
670749,Z2BR,395236,1996-01-31,20735.0
519080,Z2BR,395002,1996-01-31,103644.0
605836,Z2BR,394632,1996-01-31,38137.0
...,...,...,...,...
65304,Z2BR,395205,2021-05-31,154572.0
64438,Z2BR,394488,2021-05-31,58797.0
64417,Z2BR,394579,2021-05-31,100221.0
64474,Z2BR,394447,2021-05-31,122964.0


In [206]:
metro_3br = metro_homes[metro_homes.indicator_id == 'Z3BR']
metro_3br.sort_values(by='date', inplace=True)
metro_3br

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,indicator_id,region_id,date,value
764998,Z3BR,394949,1996-01-31,73795.0
687949,Z3BR,394669,1996-01-31,144998.0
713829,Z3BR,394875,1996-01-31,93283.0
818861,Z3BR,394556,1996-01-31,230000.0
683640,Z3BR,394475,1996-01-31,103488.0
...,...,...,...,...
66803,Z3BR,394849,2021-05-31,162608.0
68789,Z3BR,394801,2021-05-31,105781.0
69140,Z3BR,394364,2021-05-31,66896.0
66800,Z3BR,394714,2021-05-31,245144.0


In [207]:
metro_4br = metro_homes[metro_homes.indicator_id == 'Z4BR']
metro_4br.sort_values(by='date', inplace=True)
metro_4br

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,indicator_id,region_id,date,value
1176965,Z4BR,753890,1996-01-31,120535.0
1131475,Z4BR,394572,1996-01-31,104786.0
1245249,Z4BR,753865,1996-01-31,108239.0
1224486,Z4BR,394933,1996-01-31,44112.0
1174883,Z4BR,394949,1996-01-31,116969.0
...,...,...,...,...
74056,Z4BR,394500,2021-05-31,259472.0
74053,Z4BR,753865,2021-05-31,253778.0
74050,Z4BR,394409,2021-05-31,87370.0
74131,Z4BR,753916,2021-05-31,541437.0


In [208]:
metro_5br = metro_homes[metro_homes.indicator_id == 'Z5BR']
metro_5br.sort_values(by='date', inplace=True)
metro_5br

C:\ProgramData\Anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,indicator_id,region_id,date,value
281576,Z5BR,394997,1996-01-31,188033.0
262000,Z5BR,395059,1996-01-31,453298.0
358242,Z5BR,395086,1996-01-31,178269.0
255951,Z5BR,395025,1996-01-31,184235.0
310182,Z5BR,394848,1996-01-31,175141.0
...,...,...,...,...
62523,Z5BR,394988,2021-05-31,344888.0
62520,Z5BR,394941,2021-05-31,267785.0
62517,Z5BR,394859,2021-05-31,316594.0
63891,Z5BR,395236,2021-05-31,214734.0


In [209]:
metro_condo.reset_index(drop=True)

,indicator_id,region_id,date,value
0,ZCON,394619,1996-01-31,70863.0
1,ZCON,394314,1996-01-31,85074.0
2,ZCON,394528,1996-01-31,105987.0
3,ZCON,394764,1996-01-31,49046.0
4,ZCON,394463,1996-01-31,135785.0
...,...,...,...,...
184027,ZCON,394580,2021-05-31,147685.0
184028,ZCON,394709,2021-05-31,190761.0
184029,ZCON,394599,2021-05-31,180417.0
184030,ZCON,394907,2021-05-31,196293.0


In [210]:
metro_condo.to_csv('./Resources/metro_condo.csv', index=False)

In [211]:
metro_1br.reset_index(drop=True)

,indicator_id,region_id,date,value
0,Z1BR,394891,1996-01-31,42905.0
1,Z1BR,753892,1996-01-31,26374.0
2,Z1BR,394788,1996-01-31,87144.0
3,Z1BR,394829,1996-01-31,60190.0
4,Z1BR,394463,1996-01-31,99397.0
...,...,...,...,...
210181,Z1BR,394886,2021-05-31,105399.0
210182,Z1BR,394962,2021-05-31,57716.0
210183,Z1BR,394491,2021-05-31,96095.0
210184,Z1BR,394346,2021-05-31,171744.0


In [212]:
metro_1br.to_csv('./Resources/metro_1br.csv', index=False)

In [213]:
metro_2br.reset_index(drop=True)

,indicator_id,region_id,date,value
0,Z2BR,394314,1996-01-31,28575.0
1,Z2BR,394808,1996-01-31,40731.0
2,Z2BR,395236,1996-01-31,20735.0
3,Z2BR,395002,1996-01-31,103644.0
4,Z2BR,394632,1996-01-31,38137.0
...,...,...,...,...
225351,Z2BR,395205,2021-05-31,154572.0
225352,Z2BR,394488,2021-05-31,58797.0
225353,Z2BR,394579,2021-05-31,100221.0
225354,Z2BR,394447,2021-05-31,122964.0


In [214]:
metro_2br.to_csv('./Resources/metro_2br.csv', index=False)

In [215]:
metro_3br.reset_index(drop=True)

,indicator_id,region_id,date,value
0,Z3BR,394949,1996-01-31,73795.0
1,Z3BR,394669,1996-01-31,144998.0
2,Z3BR,394875,1996-01-31,93283.0
3,Z3BR,394556,1996-01-31,230000.0
4,Z3BR,394475,1996-01-31,103488.0
...,...,...,...,...
227046,Z3BR,394849,2021-05-31,162608.0
227047,Z3BR,394801,2021-05-31,105781.0
227048,Z3BR,394364,2021-05-31,66896.0
227049,Z3BR,394714,2021-05-31,245144.0


In [216]:
metro_3br.to_csv('./Resources/metro_3br.csv', index=False)

In [217]:
metro_4br.reset_index(drop=True)

,indicator_id,region_id,date,value
0,Z4BR,753890,1996-01-31,120535.0
1,Z4BR,394572,1996-01-31,104786.0
2,Z4BR,753865,1996-01-31,108239.0
3,Z4BR,394933,1996-01-31,44112.0
4,Z4BR,394949,1996-01-31,116969.0
...,...,...,...,...
225980,Z4BR,394500,2021-05-31,259472.0
225981,Z4BR,753865,2021-05-31,253778.0
225982,Z4BR,394409,2021-05-31,87370.0
225983,Z4BR,753916,2021-05-31,541437.0


In [218]:
metro_4br.to_csv('./Resources/metro_4br.csv', index=False)

In [219]:
metro_5br.reset_index(drop=True)

,indicator_id,region_id,date,value
0,Z5BR,394997,1996-01-31,188033.0
1,Z5BR,395059,1996-01-31,453298.0
2,Z5BR,395086,1996-01-31,178269.0
3,Z5BR,395025,1996-01-31,184235.0
4,Z5BR,394848,1996-01-31,175141.0
...,...,...,...,...
225020,Z5BR,394988,2021-05-31,344888.0
225021,Z5BR,394941,2021-05-31,267785.0
225022,Z5BR,394859,2021-05-31,316594.0
225023,Z5BR,395236,2021-05-31,214734.0


In [220]:
metro_5br.to_csv('./Resources/metro_5br.csv', index=False)